In [ ]:
# Python Standard Library Packages
import os
import glob
import argparse
import copy

# Community Developed Packages
import numpy as np
import astropy.units as u
import astropy.constants as c
import astropy.coordinates as aco
# import matplotlib
# matplotlib.use('Agg')
import matplotlib.pyplot as plt

# HERA Collaboration Packages
import hera_pspec as hp
import pyuvdata
from pyuvdata import UVData

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument(
    '-F',
    '--files',
    help='Designate the hdf5 files to be concatenated in time.',
    nargs='*',
    required=True)
parser.add_argument(
    '-W',
    '--wedge',
    help='Turn wedge folding on',
    action='store_true')
parser.add_argument(
    '-C',
    '--cosmo',
    help='Apply cosmological units.',
    action='store_true')
parser.add_argument(
    '-N',
    '--noise',
    help='Plot a noise estimation.',
    action='store_true')
parser.add_argument(
    '-S',
    '--savepath',
    help='Designate the path where the new hdf5 files will be saved. Default is path to data files.')

In [ ]:
"""Uncomment this code when running as .py:"""
# args = parser.parse_args()
# dfiles = np.array(sorted(args.files))

In [ ]:
"""Uncomment this code when running as .ipynb:"""
args = parser.parse_args(
    "-F /lustre/aoc/projects/hera/afortino/H1C_IDR2_1/OCRS/2458098/LSThrs_5.0_to_6.0/FREQrng_530_to_730/*.[p,x,y]?.* -C".split())

In [ ]:
"""Formatting command line arguments:"""
dfiles = sorted(glob.glob(args.files[0]))
dfiles_basename = [os.path.basename(dfile) for dfile in dfiles]
wedge = args.wedge
cosmo = args.cosmo
noise = args.noise
if args.savepath is None:
    savepath = os.path.dirname(args.files[0])
else:
    savepath = args.savepath
print 'Saving files to:\n{}'.format(savepath)

In [ ]:
"""Loading metadata:"""
# This will be deprecated once the UVPspec objects supports adding additional attributes
metadata = np.load(os.path.join(os.path.dirname(dfiles[0]), 'metadata.npz'))

FREQrng = metadata['FREQrng'].tolist()
antpos = metadata['antpos'].tolist()
blp_reds = metadata['blp_reds'].tolist()
blpairs = [(tuple(blpair[0]), tuple(blpair[1])) for blpair in metadata['blpairs'].tolist()]
bls_reds = metadata['bls_reds'].tolist()
baselines = metadata['baselines'].tolist()
norms = metadata['norms'].tolist()

LSTrng = metadata['uvd_extra_keywords'].tolist()['LSTrng'].tolist()
JD = metadata['uvd_extra_keywords'].tolist()['JD']
JDt0 = metadata['uvd_extra_keywords'].tolist()['JDt0']
JDtf = metadata['uvd_extra_keywords'].tolist()['JDtf']
numfiles = metadata['uvd_extra_keywords'].tolist()['numfiles']
ext = metadata['uvd_extra_keywords'].tolist()['ext']
xants = metadata['uvd_extra_keywords'].tolist()['xants'].tolist()

In [ ]:
"""
Order files so they are plotted like:
    pI pQ pU pV
    XX YY XY YX
"""
# Dictionary to sort the files by polarization
pol_sort = {'pI': 1, 'pQ': 2, 'pU': 3, 'pV': 5, 'XX': 5, 'YY': 6, 'XY': 7, 'YX': 8}

# Sorting function to grab the polarization and find its index from pol_sort dictionary
def pol_sort_fn(x, y):
    uvpx = hp.UVPSpec()
    uvpx.read_hdf5(x, just_meta=True)
    polx = pol_sort[pyuvdata.utils.polnum2str(uvpx.pol_array[0])]

    uvpy = hp.UVPSpec()
    uvpy.read_hdf5(y, just_meta=True)
    poly = pol_sort[pyuvdata.utils.polnum2str(uvpy.pol_array[0])]

    return int(polx - poly)

dfiles.sort(cmp=pol_sort_fn)

In [ ]:
def get_cmap(n, name='jet'):
    return plt.cm.get_cmap(name, n)

"""Making plots:"""
# Determine how many rows and columns are needed
if (len(dfiles) <= 4) or (len(dfiles) > 8):
    ncols = len(dfiles)
    nrows = 1
else:
    ncols = 4
    nrows = 2
# Initialize the axes objects
f, axes = plt.subplots(
    ncols=ncols,
    nrows=nrows,
    sharex=True,
    sharey=True,
    squeeze=False,
    figsize=(25, 10))

# Plot each file
pols = []
plt.subplots_adjust(wspace=0, hspace=0.2)
for dfile, ax in zip(dfiles, axes.flatten()):
    # Load in UVPspec objects and average the spectra along redundant baseline groups and time
    uvp = hp.UVPSpec()
    uvp.read_hdf5(dfile)

    pol = pyuvdata.utils.polnum2str(uvp.pol_array[0])
    pols.append(pol)

    times = aco.Angle((np.unique(uvp.lst_1_array)*u.rad)).hour
    total_time = np.sum(np.diff(times))
    hours = int(total_time)
    minutes = (total_time*60) % 60
    seconds = (total_time*3600) % 60
    total_time = '%d:%02d:%02d' % (hours, minutes, seconds)

    FREQrng_MHz = np.around((uvp.freq_array*u.Hz).to(u.MHz), 2)
    Central_Frequency = np.median(FREQrng_MHz)

    uvp.average_spectra(blpair_groups=blp_reds.values(), time_avg=True)

    Tsys = 360
    pspec_noise = uvp.generate_noise_spectra(0, pol, Tsys)

    # Format data array
    data = uvp.data_array[0][:, :, 0]
    if wedge:
        uvp.fold_spectra()
        data = uvp.data_array[0][:, data.shape[1] // 2:, 0]

    # Format x-axis values and set x-axis label
    delays = (uvp.get_dlys(0)*1e9).tolist()
    k_paras = uvp.get_kparas(0).tolist()
    if cosmo:
        x_axis = k_paras
        ax2 = ax.twiny()
        x_axis_2 = delays
    else:
        x_axis = delays
        x_axis_2 = None

    if wedge:
        x_axis.insert(0, 0) 
        if x_axis_2:
            x_axis_2.insert(0, 0) 
    ax.set_xlim((x_axis[0], x_axis[-1]))
    if x_axis_2:
        ax2.set_xlim((x_axis_2[0], x_axis_2[-1]))

    # Plot each redundant baseline power spectra
    cmap = get_cmap(data.shape[0])
    for i, (group, noise_key) in enumerate(zip(data, pspec_noise)):

        if noise:
            ax.semilogy(
                x_axis,
                np.abs(pspec_noise[noise_key][0, :]),
                c=cmap(i),
                lw=1,
                ls='--')

        ax.semilogy(
            x_axis,
            np.abs(group),
            c=cmap(i),
            label='{norm}m ({bls} bls)'.format(norm=norms[i], bls=len(bls_reds[norms[i]])),
            lw=1,
            ls='-')

    if not noise:
        ax.set_ylim((1e7, 1e16))

    ax.grid()
    ax.set_title(pol+'\n', fontsize=15)

if cosmo:
    f.text(0.5, 0.06, r'$k_{\parallel}\ [h\ Mpc^{-1}]$', fontsize=20, ha='center')
    f.text(0.5, 0.94, r'$\tau\ [ns]$', fontsize=20, ha='center')
else:
    f.text(0.5, 0.06, r'$\tau\ [ns]$', fontsize=20, ha='center')

if ('C' in ext) or ('K' in ext):
    f.text(0.09, 0.5, r'$P(k_{\parallel})\ [mK^2h^{-3}Mpc^3]$', fontsize=20, va='center', rotation='vertical')
else:
    f.text(0.09, 0.5, r'$P(\tau)\ [mK^2]$', fontsize=20, va='center', rotation='vertical')

"""Information"""
f.text(0.15, 0.94, 'Total Time Integrated: {}\nLST Hour from {} to {}\nJD {} from file .{} to .{}'.format(
    total_time, np.round(times[0], 2), np.round(times[-1], 2), JD, JDt0, JDtf), ha='left', fontsize=10)
f.text(0.85, 0.94, 'Frequency Range: Channels {} to {}\nFrequency Range: {} to {}\nCentral Frequency: {}'.format(
    FREQrng[0], FREQrng[-1], FREQrng_MHz[0], FREQrng_MHz[-1], Central_Frequency), ha='right', fontsize=10)
f.text(0.5, 0.985, 'Excluded Antenna: {}'.format(xants), fontsize=10, ha='center')    

if noise:
    f.text(0.5, 0.968, 'Dotted Lines: Estimation of Noise at System Temperature of {}K'.format(Tsys), fontsize=10, ha='center')    
if (len(dfiles) <= 4) or (len(dfiles) > 8):
    ax.legend(loc='upper right', ncol=2 )

In [ ]:
pols = '_'.join(pols)
tag = ['1D']
if wedge:
    tag.append('wdg')
else:
    tag.append('pf')

if cosmo:
    tag.append('cosmo')
else:
    tag.append('std')

if noise:
    tag.append('noise')
tag = '_'.join(tag)

filename = os.path.join(savepath, 'zen.{JD}.{JDt0}_{JDtf}.{pols}.HH.{tag}.{ext}.pdf'.format(
    JD=JD,
    JDt0=JDt0,
    JDtf=JDtf,
    pols=pols,
    tag=tag,
    ext=ext))

print 'Saving:'
print filename
# plt.savefig(filename)